# Function Calling/Tool Use


## Just a quick definition

Function calling in the context of Large Language Models (LLMs) refers to the ability of the model to invoke external functions or tools during its response generation. Instead of only generating text, the LLM can recognize when a task requires external data or computation, call a predefined function with the appropriate arguments, and then use the function's output to continue its response.

Think of having a super smart robot that you can only speak with. Pretty useless at doing anything aside from talking. Now imagine you have given that robot a hammer and some nails. It can now put up that wall painting that's been waiting forever to be hung :D

Except, the way we as AI Engineers give AI tools is by defining python functions and describing that function in detail to the LLM so it knows what tools it has access to, what each tool can do, what are its inputs and expected outputs.

Just remember, a *tool* in its simplest form is just a *python function* that you have defined. It can be as simple as a calculator function or something like being able to call external APIs.

![tool-use](../images/tool-use.png)

*Image courtesy of [API Deck](https://www.apideck.com/blog/llm-tool-use-and-function-calling)*

## Step 1: Import libraries

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

## Step 2: Define a tool

In [2]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [3]:
get_weather(24.343627, 54.497922)

23.9

## Step 3: Call a chat model normally

In [4]:
client = OpenAI()

# The chat.completions endpoint is the traditional way to get responses from models like ChatGPT, requiring you to manage conversation 
# history by sending the full message list with each request. In contrast, the newer Responses API is stateful, meaning it manages 
# history internally based on a conversation ID, which simplifies development and is optimized for agent-like workflows with built-in 
# tools like web search and function calling. While both can be used for chat, the Responses API is recommended for new projects due 
# to its built-in features and simpler history management, though chat.completions will still be supported. Note the input= parameter 
# in the Responses API

response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {"role": "user", "content": "What is the weather like in Abu Dhabi now?"}
    ]
)

print(response.output_text)

I'm unable to provide real-time weather updates. For the most current weather in Abu Dhabi, I recommend checking a reliable weather website or app such as Weather.com, AccuWeather, or a local weather service.


In [6]:
response.__dict__
#Note the response id at the top and the output. Also note the tools  object which is empty as we have not tool in use yet.


{'id': 'resp_0b5dab7e9760cd54006920eed9e40c81a2b6532660786fea97',
 'created_at': 1763765977.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_0b5dab7e9760cd54006920eeda8b8081a28bb71cba0eb8b150', content=[ResponseOutputText(annotations=[], text="I'm unable to provide real-time weather updates. For the most current weather in Abu Dhabi, I recommend checking a reliable weather website or app such as Weather.com, AccuWeather, or a local weather service.", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'previous_response_id': None,
 'prompt': None,
 'prompt_cache_key': None,
 'reasoning': Reasoning(effort=None,

In [7]:
response.output[0].__dict__
# Looking at the type = message. This will change to function when we use tools.

{'id': 'msg_0b5dab7e9760cd54006920eeda8b8081a28bb71cba0eb8b150',
 'content': [ResponseOutputText(annotations=[], text="I'm unable to provide real-time weather updates. For the most current weather in Abu Dhabi, I recommend checking a reliable weather website or app such as Weather.com, AccuWeather, or a local weather service.", type='output_text', logprobs=[])],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

## Step 4: Define the input schema for our tool

Lets try giving our `get_weather` function as a tool to our AI.

We will first need to define the tool in a format OpenAI can understand

In [8]:
tools = [{
    'type': 'function',
    'name': 'get_weather',
    'description': 'Fetch the current weather for a specific location',
    'parameters': {
        'type': 'object',
        'properties': {
            'latitude': {'type': 'number'},
            'longitude': {'type': 'number'}
        },
        'required': ['latitude', 'longitude'],
        'additionalProperties': False
    },
    'strict': True
}]

## Step 5: Pass the tool schema over to the model

In [9]:
input_messages = [{"role": "user", "content": "What is the weather like in Abu Dhabi now?"}]

In [10]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
)

In [11]:
print(response.output_text)
# there is no output yet as the model has decided to call a function

In [12]:
response.__dict__
# notice the output is a ResponseFunctionCall object.

{'id': 'resp_02141e41f4dcc3bb006920ef2141148192a0ace01ee8523628',
 'created_at': 1763766049.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseFunctionToolCall(arguments='{"latitude":24.4539,"longitude":54.3773}', call_id='call_b5c0aDreSDbhw4ypQCZHkrSv', name='get_weather', type='function_call', id='fc_02141e41f4dcc3bb006920ef2250bc81928d5e011604a9d164', status='completed')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Fetch the current weather for a specific location')],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 'max_output_tokens': None,
 'max_tool_calls':

In [13]:
response.output[0].__dict__
# Inspecting the output object for more details

{'arguments': '{"latitude":24.4539,"longitude":54.3773}',
 'call_id': 'call_b5c0aDreSDbhw4ypQCZHkrSv',
 'name': 'get_weather',
 'type': 'function_call',
 'id': 'fc_02141e41f4dcc3bb006920ef2250bc81928d5e011604a9d164',
 'status': 'completed'}

## Step 6: Format the tool call response from the LLM

In [ ]:
import json
# Extracting the tool call and its arguments
tool_call = response.output[0]
# the arguments are in JSON format
args = json.loads(tool_call.arguments)

In [15]:
print(tool_call)

ResponseFunctionToolCall(arguments='{"latitude":24.4539,"longitude":54.3773}', call_id='call_b5c0aDreSDbhw4ypQCZHkrSv', name='get_weather', type='function_call', id='fc_02141e41f4dcc3bb006920ef2250bc81928d5e011604a9d164', status='completed')


In [16]:
print(args)

{'latitude': 24.4539, 'longitude': 54.3773}


## Step 7: Pass on the tool call arguments to our tool/python function

We now need to pass on the arguments received by the model to our python function or tool

In [17]:
result = get_weather(args['latitude'], args['longitude'])
result

23.9

## Step 8: Append the response of the tool into the message list

In [ ]:
# input messages already has the original question (what is the weather like in Abu Dhabi now? )
# Now add the definition of the tool call chatgpt asked us to call
input_messages.append(tool_call)
# Then add the output of the tool call. The call id links the output to the specific tool call
input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

In [ ]:
# use pretty print to see the input messages a bit clearer
from pprint import pprint
pprint(input_messages)

[{'content': 'What is the weather like in Abu Dhabi now?', 'role': 'user'},
 ResponseFunctionToolCall(arguments='{"latitude":24.4539,"longitude":54.3773}', call_id='call_b5c0aDreSDbhw4ypQCZHkrSv', name='get_weather', type='function_call', id='fc_02141e41f4dcc3bb006920ef2250bc81928d5e011604a9d164', status='completed'),
 {'call_id': 'call_b5c0aDreSDbhw4ypQCZHkrSv',
  'output': '23.9',
  'type': 'function_call_output'}]


## Step 9: Pass the message list into the model

In [20]:
response_2 = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
) 

In [21]:
print(response_2.output_text)

The current temperature in Abu Dhabi is approximately 23.9°C.


In [22]:
response_2.__dict__

{'id': 'resp_02141e41f4dcc3bb006920ef9c863081928f06a807c1b37d65',
 'created_at': 1763766172.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_02141e41f4dcc3bb006920ef9cf6008192916c9384b1601c0b', content=[ResponseOutputText(annotations=[], text='The current temperature in Abu Dhabi is approximately 23.9°C.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Fetch the current weather for a specific location')],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 '

# Resources:

- [OpenAIs Function Calling Guide](https://platform.openai.com/docs/guides/function-calling?api-mode=responses)
- [Anthropics Tool Use Guide with Claude](https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/overview)
- [Function Calling with Gemini API](https://ai.google.dev/gemini-api/docs/function-calling?example=meeting)

<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>

  <br>
  Now that you've seen how to define a Python function as a tool and connect it to an LLM, it's time to get creative!

  <b>Pick your challenge</b><br>
  <b>1. Wikipedia Article Summarizer</b><br>
  - Fetch a wikipedia article you are interested in learning about and have an LLM summarize it<br>
  - <a href="https://pypi.org/project/Wikipedia-API/">Wikipedia API</a><br>

  <b>2. News Summarizer</b><br>
  - Fetch latest headlines or news by topic.<br>
  - <a href="https://newsapi.org">NewsAPI</a><br>
  - <a href="https://currentsapi.services/en/docs/">CurrentsAPI</a><br>

  <b>3. Stock Market Data</b><br>
  - Get real-time stock prices or company info<br>
  - <a href="https://www.alphavantage.co/">Alpha Vantage</a><br>
  - <a href="https://pypi.org/project/yfinance/">yfinance</a><br>

  <b>4. Movie Info</b><br>
  - Get movie ratings, cast, plot summaries.<br>
  - <a href="https://imdbapi.dev/">IMDb</a><br>
  - <a href="https://www.omdbapi.com/">OMDb API</a><br>

  <b>5. NASA API</b><br>
  - Astronomy facts, country data, etc.<br>
  - <a href="https://api.nasa.gov/">NASA APIs</a><br>

  <b>6. Your own custom tool</b><br>
  - Think of a real-world use case where an LLM could benefit from calling a custom function.<br>

  <hr>

  <b>Tips</b>:<br>
  - Use clear function names and docstrings.<br>
  - Handle input arguments and outputs carefully.<br>
  - Print the LLM's tool call and your function's output in a readable way.<br>

  Be sure to place your submissions in <code>part1-fundamentals/community-contributions/&lt;your-name&gt;</code><br>

  I'm super excited to see what you come up with :D
</div>